In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

import time
import re
from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
df = pd.read_csv('url_completed_test.csv')

data_list = []
user_review_id = {} # 유저 정보
review_json = {} # 리뷰 
image_json = {} # 리뷰 이미지
menu_json = {} # 메뉴


chromedriver = 'C:/Users/lim98/Desktop/chromedriver_win32/chromedriver'
main_driver = webdriver.Chrome(chromedriver)
review_driver = webdriver.Chrome(chromedriver)
menu_driver = webdriver.Chrome(chromedriver)

for i in range(len(df)):
    
    print('======================================================') 
    print(str(i)+'번째 식당') 
    
    main_driver.get(df['naver_map_url'][i]) 
    thisurl = df['naver_map_url'][i]
    review_driver.get(thisurl + "/review/visitor")
    menu_driver.get(thisurl + "/menu/list")
    time.sleep(3) 
    
    while True: 
        try: 
            time.sleep(1) 
            review_driver.find_element_by_tag_name('body').send_keys(Keys.END) 
            time.sleep(2) 
            
            review_driver.find_element_by_css_selector('div#app-root.place_didmount > div > div.BXtr_.tAvTy > div > div:nth-child(7) > div:nth-child(2) > div:nth-child(3) > div.lfH3O > a').send_keys(Keys.ENTER)
            time.sleep(2)
            review_driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(1)
                
        except NoSuchElementException:
            break
            
    html1 = main_driver.page_source
    soup1 = BeautifulSoup(html1, 'lxml')
    html2 = review_driver.page_source
    soup2 = BeautifulSoup(html2, 'lxml')
    html3 = menu_driver.page_source
    soup3 = BeautifulSoup(html3, 'lxml')
    time.sleep(2)
    
    try:
        restaurant_name = soup1.find_all('span', attrs = {'class':'Fc1rA'})[0].text
    except:
        restaurant_name = 'none'
    
    try: 
        restaurant_classification = soup1.find_all('span', attrs = {'class':'DJJvD'})[0].text 
    except: 
        restaurant_classification = 'none'
    
    try:
        road_address = soup1.find_all('span', attrs = {'class':'LDgIH'})[0].text
    except:
        road_address = 'none'
        
    try:
        tel = soup1.find_all('span', attrs = {'class':'xlx7Q'})[0].text
    except:
        tel = 'none'
    try:
        rating = soup1.find_all('span', attrs = {'class':'PXMot LXIwF'}).text
    except:
        rating = 'none'

    user_review_id[restaurant_name] = {}
    review_json[restaurant_name] = {} 
    image_json[restaurant_name] = {} 
    menu_json[restaurant_name] = {}
    
    try:
        menu_board = soup3.find_all('li', attrs = {'class':'E2jtL'})
        assert len(menu_board) != 0
        for i in range(len(menu_board)):
            menu = menu_board[i].find('span', attrs = {'class':'lPzHi'}).text + " " + menu_board[i].find('div', attrs = {'class' : 'GXS1X'}).text
            menu_json[restaurant_name] = menu
            
    except:
        menu_json[restaurant_name] = ""
                                                                                                                                                                                     
    try:
        one_review = soup2.find_all('li', attrs = {'class':'YeINN'})
        assert len(one_review) != 0
        review_num = len(one_review)
    
        for i in range(len(one_review)):
    
            user_url = one_review[i].find('div', attrs = {'class':'Lia3P'}).find('a').get('href')
            user_code = re.findall(r"my/(\w+)", user_url)[0]
        
            res_code = re.findall(r"restaurant/(\d+)", thisurl)[0]
            review_id = str(res_code) + "_" + user_code
            
            date = one_review[i].find_all('span', attrs = {'class':'place_blind'})[-1].text
            
            try:
                review_content = one_review[i].find('span', attrs = {'class':'zPfVt'}).text
            except: 
                review_content = "" 
                
            # image 내용
            sliced_soup = one_review[i].find('div', attrs = {'class':'Mmpzx'})
            
            if (sliced_soup != None): 
                sliced_soup = sliced_soup.find('div', attrs={'class':'K0PDV _img fKa0W'})
 
                try: 
                    img_url = 'https://search.pstatic.net/common/?autoRotate=true&quality=95&type=w750&src='+ re.findall(r'src=(.*jpeg)', str(sliced_soup))[0] 
                    
                except : 
                    if (len(re.findall(r'src=(.*jpg)', str(sliced_soup)))!= 0): 
                        img_url = 'https://search.pstatic.net/common/?autoRotate=true&quality=95&type=w750&src='+ re.findall(r'src=(.*jpg)', str(sliced_soup))[0]  
                    elif (len(re.findall(r'src=(.*png)', str(sliced_soup)))!= 0): 
                        img_url = 'https://search.pstatic.net/common/?autoRotate=true&quality=95&type=w750&src='+ re.findall(r'src=(.*png)', str(sliced_soup))[0]  
                    else : 
                        img_url = ""
            
            else: 
                img_url = "" 
            
            # user_review_id
            user_review_id[restaurant_name][user_code] = review_id 
            
            # review_json 
            review_json[restaurant_name][review_id] = review_content 
            
            # image_json 
            image_json[restaurant_name][review_id] = img_url 
            
            naver_review = user_code, restaurant_name, restaurant_classification, road_address, tel, rating, date
            data_list.append(naver_review) 
    
    except:
        none_review = "네이버 리뷰 없음" 
        print(none_review) 
        review_num = 0 
        
        naver_review = user_code, restaurant_name, restaurant_classification, road_address, tel, rating, none_review
        data_list.append(naver_review)

print('\n')   

0번째 식당
1번째 식당
2번째 식당
3번째 식당
4번째 식당
5번째 식당
6번째 식당
7번째 식당
8번째 식당
9번째 식당
10번째 식당
11번째 식당
12번째 식당
13번째 식당
14번째 식당
15번째 식당
16번째 식당
17번째 식당
18번째 식당
19번째 식당
20번째 식당
21번째 식당
22번째 식당
23번째 식당
24번째 식당
25번째 식당
26번째 식당
27번째 식당
28번째 식당
29번째 식당
30번째 식당
31번째 식당
32번째 식당
33번째 식당
34번째 식당
35번째 식당
36번째 식당
37번째 식당
38번째 식당
39번째 식당
40번째 식당
41번째 식당
42번째 식당
43번째 식당
44번째 식당
45번째 식당
46번째 식당
47번째 식당
48번째 식당
49번째 식당
50번째 식당
51번째 식당
52번째 식당
53번째 식당
54번째 식당
55번째 식당
56번째 식당
57번째 식당
58번째 식당
59번째 식당
60번째 식당
61번째 식당
62번째 식당
63번째 식당
64번째 식당
65번째 식당
66번째 식당
67번째 식당
68번째 식당
69번째 식당
70번째 식당
71번째 식당
72번째 식당
73번째 식당
74번째 식당
75번째 식당
76번째 식당
77번째 식당
78번째 식당
79번째 식당
80번째 식당
81번째 식당
82번째 식당
83번째 식당
84번째 식당
85번째 식당
86번째 식당
87번째 식당
88번째 식당
89번째 식당
90번째 식당
91번째 식당
92번째 식당
93번째 식당
94번째 식당
95번째 식당
96번째 식당
97번째 식당
98번째 식당
99번째 식당
100번째 식당
101번째 식당
102번째 식당
103번째 식당
104번째 식당
105번째 식당
106번째 식당
107번째 식당
108번째 식당
109번째 식당
110번째 식당
111번째 식당
112번째 식당
113번째 식당
114번째 식당
115번째 식당
116번째 식당
117번째 식당
118번째 식당
119번째 식당
120번째 식당
121번째 식당
122번째 식당
123

130번째 식당
131번째 식당
132번째 식당
133번째 식당
134번째 식당
135번째 식당
136번째 식당
137번째 식당
138번째 식당
139번째 식당
140번째 식당
141번째 식당
142번째 식당
143번째 식당
144번째 식당
145번째 식당
146번째 식당
147번째 식당
148번째 식당
149번째 식당
150번째 식당
151번째 식당
152번째 식당
153번째 식당
154번째 식당
155번째 식당
156번째 식당
157번째 식당
158번째 식당
159번째 식당
160번째 식당
161번째 식당
162번째 식당
163번째 식당
164번째 식당
165번째 식당
166번째 식당
167번째 식당
168번째 식당
169번째 식당
170번째 식당
171번째 식당
172번째 식당
173번째 식당
174번째 식당
175번째 식당
176번째 식당
177번째 식당
178번째 식당
179번째 식당
180번째 식당
181번째 식당
182번째 식당
183번째 식당
184번째 식당
185번째 식당
186번째 식당
187번째 식당
188번째 식당
189번째 식당
190번째 식당
191번째 식당
192번째 식당
193번째 식당
194번째 식당
195번째 식당
196번째 식당
197번째 식당
198번째 식당
199번째 식당
200번째 식당
201번째 식당
202번째 식당
203번째 식당
204번째 식당
205번째 식당
206번째 식당
207번째 식당
208번째 식당
209번째 식당
210번째 식당
211번째 식당
212번째 식당
213번째 식당
214번째 식당
215번째 식당
216번째 식당
217번째 식당
218번째 식당
219번째 식당
220번째 식당
221번째 식당
222번째 식당
223번째 식당
224번째 식당
225번째 식당
226번째 식당
227번째 식당
228번째 식당
229번째 식당
230번째 식당
231번째 식당
232번째 식당
233번째 식당
234번째 식당
235번째 식당
236번째 식당
237번째 식당
238번째 식당
239번째 식당
240번째 식당
2

259번째 식당
260번째 식당
261번째 식당
262번째 식당
263번째 식당
264번째 식당
265번째 식당
266번째 식당
267번째 식당
268번째 식당
269번째 식당
270번째 식당
271번째 식당
272번째 식당
273번째 식당
274번째 식당
275번째 식당
276번째 식당
277번째 식당
278번째 식당
279번째 식당
280번째 식당
281번째 식당
282번째 식당
283번째 식당
284번째 식당
285번째 식당
286번째 식당
287번째 식당
288번째 식당
289번째 식당
290번째 식당
291번째 식당
292번째 식당
293번째 식당
294번째 식당
295번째 식당
296번째 식당
297번째 식당
298번째 식당
299번째 식당
300번째 식당
301번째 식당
302번째 식당
303번째 식당
304번째 식당
305번째 식당
306번째 식당
307번째 식당
308번째 식당
309번째 식당
310번째 식당
311번째 식당
312번째 식당
313번째 식당
314번째 식당
315번째 식당
316번째 식당
317번째 식당
318번째 식당
319번째 식당
320번째 식당
321번째 식당
322번째 식당
323번째 식당
324번째 식당
325번째 식당
326번째 식당
327번째 식당
328번째 식당
329번째 식당
330번째 식당
331번째 식당
332번째 식당
333번째 식당
334번째 식당
335번째 식당
336번째 식당
337번째 식당
338번째 식당
339번째 식당
340번째 식당
341번째 식당
342번째 식당
343번째 식당
344번째 식당
345번째 식당
346번째 식당
347번째 식당
348번째 식당
349번째 식당
350번째 식당
351번째 식당
352번째 식당
353번째 식당
354번째 식당
355번째 식당
356번째 식당
357번째 식당
358번째 식당
359번째 식당
360번째 식당
361번째 식당
362번째 식당
363번째 식당
364번째 식당
365번째 식당
366번째 식당
367번째 식당
368번째 식당
369번째 식당
3

387번째 식당
388번째 식당
389번째 식당
390번째 식당
391번째 식당
392번째 식당
393번째 식당
394번째 식당
395번째 식당
396번째 식당
397번째 식당
398번째 식당
399번째 식당
400번째 식당
401번째 식당
402번째 식당
403번째 식당
404번째 식당
405번째 식당
406번째 식당
407번째 식당
408번째 식당
409번째 식당
410번째 식당
네이버 리뷰 없음
411번째 식당
412번째 식당
413번째 식당
414번째 식당
415번째 식당
416번째 식당
417번째 식당
418번째 식당
419번째 식당
420번째 식당
421번째 식당
422번째 식당
423번째 식당
424번째 식당
425번째 식당
426번째 식당
427번째 식당
428번째 식당
429번째 식당
430번째 식당
431번째 식당
432번째 식당
433번째 식당
네이버 리뷰 없음
434번째 식당
435번째 식당
436번째 식당
437번째 식당
438번째 식당
439번째 식당
440번째 식당
441번째 식당
442번째 식당
443번째 식당
444번째 식당
445번째 식당
446번째 식당
447번째 식당
448번째 식당
449번째 식당
네이버 리뷰 없음
450번째 식당
451번째 식당
452번째 식당
453번째 식당
454번째 식당
455번째 식당
456번째 식당
457번째 식당
458번째 식당
459번째 식당
네이버 리뷰 없음
460번째 식당
네이버 리뷰 없음
461번째 식당
462번째 식당
463번째 식당
464번째 식당
465번째 식당
네이버 리뷰 없음
466번째 식당
네이버 리뷰 없음
467번째 식당
468번째 식당
469번째 식당
470번째 식당
471번째 식당
472번째 식당
473번째 식당
474번째 식당
475번째 식당
476번째 식당
477번째 식당
478번째 식당
479번째 식당
480번째 식당
481번째 식당
482번째 식당
483번째 식당
484번째 식당
485번째 식당
486번째 식당
487번째 식당
488번째 식당
489번째 식당
490

514번째 식당
515번째 식당
516번째 식당
517번째 식당
518번째 식당
519번째 식당
520번째 식당
521번째 식당
522번째 식당
523번째 식당
524번째 식당
525번째 식당
526번째 식당
527번째 식당
528번째 식당
529번째 식당
530번째 식당
531번째 식당
532번째 식당
533번째 식당
534번째 식당
535번째 식당
536번째 식당
537번째 식당
538번째 식당
539번째 식당
540번째 식당
541번째 식당
542번째 식당
543번째 식당
544번째 식당
545번째 식당
546번째 식당
547번째 식당
548번째 식당
549번째 식당
550번째 식당
551번째 식당
552번째 식당
553번째 식당
554번째 식당
555번째 식당
556번째 식당
557번째 식당
558번째 식당
559번째 식당
560번째 식당
561번째 식당
562번째 식당
563번째 식당
564번째 식당
565번째 식당
566번째 식당
567번째 식당
568번째 식당
569번째 식당
570번째 식당
571번째 식당
572번째 식당
573번째 식당
574번째 식당
575번째 식당
576번째 식당
577번째 식당
578번째 식당
579번째 식당
580번째 식당
581번째 식당
582번째 식당
583번째 식당
584번째 식당
585번째 식당
586번째 식당
587번째 식당
588번째 식당
589번째 식당
590번째 식당
591번째 식당
592번째 식당
593번째 식당
594번째 식당
595번째 식당
596번째 식당
597번째 식당
598번째 식당
599번째 식당
600번째 식당
601번째 식당
602번째 식당
603번째 식당
604번째 식당
605번째 식당
606번째 식당
607번째 식당
608번째 식당
609번째 식당
610번째 식당
611번째 식당
612번째 식당
613번째 식당
614번째 식당
615번째 식당
616번째 식당
617번째 식당
618번째 식당
619번째 식당
620번째 식당
621번째 식당
622번째 식당
623번째 식당
624번째 식당
6

...

2538번째 식당
2539번째 식당
2540번째 식당
2541번째 식당
2542번째 식당
2543번째 식당
2544번째 식당
2545번째 식당
2546번째 식당
2547번째 식당
2548번째 식당
2549번째 식당
2550번째 식당
2551번째 식당
네이버 리뷰 없음
2552번째 식당
2553번째 식당
2554번째 식당
2555번째 식당
2556번째 식당
2557번째 식당
2558번째 식당
2559번째 식당
2560번째 식당
2561번째 식당
2562번째 식당
2563번째 식당
2564번째 식당
2565번째 식당
2566번째 식당
2567번째 식당
2568번째 식당
2569번째 식당
2570번째 식당
2571번째 식당
2572번째 식당
2573번째 식당
2574번째 식당
2575번째 식당
2576번째 식당
2577번째 식당
2578번째 식당
2579번째 식당
2580번째 식당
2581번째 식당
2582번째 식당
2583번째 식당
2584번째 식당
2585번째 식당
2586번째 식당
2587번째 식당
2588번째 식당
2589번째 식당
2590번째 식당
2591번째 식당
2592번째 식당
2593번째 식당
2594번째 식당
2595번째 식당
2596번째 식당
2597번째 식당
2598번째 식당
2599번째 식당
2600번째 식당
2601번째 식당
2602번째 식당
2603번째 식당
2604번째 식당
2605번째 식당
2606번째 식당
2607번째 식당
2608번째 식당
2609번째 식당
2610번째 식당
2611번째 식당
2612번째 식당
2613번째 식당
2614번째 식당
2615번째 식당
2616번째 식당
2617번째 식당
2618번째 식당
2619번째 식당
2620번째 식당
2621번째 식당
2622번째 식당
2623번째 식당
2624번째 식당
2625번째 식당
2626번째 식당
2627번째 식당
2628번째 식당
2629번째 식당
2630번째 식당
2631번째 식당
2632번째 식당
2633번째 식당
2634번째 식당
2635번째 식당
2636번째 식당


2664번째 식당
네이버 리뷰 없음
2665번째 식당
2666번째 식당
2667번째 식당
2668번째 식당
2669번째 식당
2670번째 식당
2671번째 식당
2672번째 식당
2673번째 식당
2674번째 식당
2675번째 식당
2676번째 식당
2677번째 식당
2678번째 식당
2679번째 식당
2680번째 식당
2681번째 식당
2682번째 식당
2683번째 식당
2684번째 식당
2685번째 식당
2686번째 식당
네이버 리뷰 없음
2687번째 식당
2688번째 식당
2689번째 식당
2690번째 식당
2691번째 식당
2692번째 식당
2693번째 식당
2694번째 식당
2695번째 식당
2696번째 식당
2697번째 식당
2698번째 식당
2699번째 식당
2700번째 식당
2701번째 식당
2702번째 식당
2703번째 식당
2704번째 식당
2705번째 식당
2706번째 식당
2707번째 식당
2708번째 식당
2709번째 식당
2710번째 식당
2711번째 식당
2712번째 식당
2713번째 식당
2714번째 식당
2715번째 식당
2716번째 식당
2717번째 식당
2718번째 식당
2719번째 식당
2720번째 식당
2721번째 식당
2722번째 식당
2723번째 식당
2724번째 식당
2725번째 식당
2726번째 식당
2727번째 식당
2728번째 식당
2729번째 식당
2730번째 식당
2731번째 식당
2732번째 식당
2733번째 식당
2734번째 식당
2735번째 식당
2736번째 식당
2737번째 식당
2738번째 식당
2739번째 식당
2740번째 식당
2741번째 식당
2742번째 식당
2743번째 식당
2744번째 식당
2745번째 식당
2746번째 식당
2747번째 식당
2748번째 식당
2749번째 식당
2750번째 식당
2751번째 식당
2752번째 식당
2753번째 식당
2754번째 식당
2755번째 식당
2756번째 식당
2757번째 식당
2758번째 식당
2759번째 식당
2760번째 식당
2761번째 식당


2790번째 식당
2791번째 식당
2792번째 식당
2793번째 식당
2794번째 식당
2795번째 식당
2796번째 식당
2797번째 식당
2798번째 식당
2799번째 식당
2800번째 식당
2801번째 식당
2802번째 식당
2803번째 식당
2804번째 식당
2805번째 식당
2806번째 식당
2807번째 식당
2808번째 식당
2809번째 식당
2810번째 식당
2811번째 식당
2812번째 식당
2813번째 식당
2814번째 식당
2815번째 식당
2816번째 식당
2817번째 식당
2818번째 식당
2819번째 식당
2820번째 식당
2821번째 식당
2822번째 식당
2823번째 식당
2824번째 식당
2825번째 식당
2826번째 식당
2827번째 식당
2828번째 식당
2829번째 식당
2830번째 식당
2831번째 식당
2832번째 식당
2833번째 식당
2834번째 식당
2835번째 식당
2836번째 식당
2837번째 식당
2838번째 식당
2839번째 식당
2840번째 식당
2841번째 식당
2842번째 식당
2843번째 식당
2844번째 식당
2845번째 식당
2846번째 식당
2847번째 식당
2848번째 식당
2849번째 식당
2850번째 식당
2851번째 식당
2852번째 식당
2853번째 식당
2854번째 식당
2855번째 식당
2856번째 식당
2857번째 식당
2858번째 식당
2859번째 식당
2860번째 식당
2861번째 식당
2862번째 식당
2863번째 식당
2864번째 식당
2865번째 식당
2866번째 식당
2867번째 식당
2868번째 식당
2869번째 식당
2870번째 식당
2871번째 식당
2872번째 식당
2873번째 식당
2874번째 식당
2875번째 식당
2876번째 식당
2877번째 식당
2878번째 식당
2879번째 식당
2880번째 식당
2881번째 식당
2882번째 식당
2883번째 식당
2884번째 식당
2885번째 식당
2886번째 식당
2887번째 식당
2888번째 식당
2889번째 식당


2917번째 식당
2918번째 식당
2919번째 식당
2920번째 식당
2921번째 식당
2922번째 식당
네이버 리뷰 없음
2923번째 식당
2924번째 식당
2925번째 식당
2926번째 식당
2927번째 식당
2928번째 식당
2929번째 식당
2930번째 식당
2931번째 식당
2932번째 식당
2933번째 식당
2934번째 식당
2935번째 식당
2936번째 식당
2937번째 식당
2938번째 식당
2939번째 식당
2940번째 식당
2941번째 식당
2942번째 식당
2943번째 식당
2944번째 식당
2945번째 식당
2946번째 식당
2947번째 식당
2948번째 식당
2949번째 식당
2950번째 식당
2951번째 식당
2952번째 식당
2953번째 식당
2954번째 식당
2955번째 식당
2956번째 식당
2957번째 식당
2958번째 식당
2959번째 식당
2960번째 식당
2961번째 식당
2962번째 식당
2963번째 식당
2964번째 식당
2965번째 식당
2966번째 식당
2967번째 식당
2968번째 식당
2969번째 식당
2970번째 식당
2971번째 식당
2972번째 식당
2973번째 식당
2974번째 식당
2975번째 식당
2976번째 식당
2977번째 식당
2978번째 식당
2979번째 식당
2980번째 식당
2981번째 식당
2982번째 식당
2983번째 식당
2984번째 식당
2985번째 식당
2986번째 식당
2987번째 식당
2988번째 식당
2989번째 식당
2990번째 식당
2991번째 식당
2992번째 식당
2993번째 식당
2994번째 식당
2995번째 식당
2996번째 식당
2997번째 식당
2998번째 식당
2999번째 식당
3000번째 식당
3001번째 식당
3002번째 식당
3003번째 식당
3004번째 식당
3005번째 식당
3006번째 식당
3007번째 식당
3008번째 식당
3009번째 식당
3010번째 식당
3011번째 식당
3012번째 식당
3013번째 식당
3014번째 식당
3015번째 식당


3043번째 식당
3044번째 식당
3045번째 식당
3046번째 식당
3047번째 식당
3048번째 식당
3049번째 식당
3050번째 식당
3051번째 식당
3052번째 식당
3053번째 식당
3054번째 식당
3055번째 식당
3056번째 식당
3057번째 식당
3058번째 식당
3059번째 식당
3060번째 식당
3061번째 식당
3062번째 식당
3063번째 식당
3064번째 식당
3065번째 식당
3066번째 식당
3067번째 식당
3068번째 식당
3069번째 식당
3070번째 식당
3071번째 식당
3072번째 식당
3073번째 식당
3074번째 식당
3075번째 식당
3076번째 식당
3077번째 식당
3078번째 식당
3079번째 식당
3080번째 식당
3081번째 식당
3082번째 식당
3083번째 식당
3084번째 식당
3085번째 식당
3086번째 식당
3087번째 식당
3088번째 식당
3089번째 식당
3090번째 식당
3091번째 식당
3092번째 식당
3093번째 식당
3094번째 식당
3095번째 식당
3096번째 식당
3097번째 식당
3098번째 식당
3099번째 식당
3100번째 식당
3101번째 식당
3102번째 식당
3103번째 식당
3104번째 식당
3105번째 식당
3106번째 식당
3107번째 식당
3108번째 식당
3109번째 식당
3110번째 식당
3111번째 식당
3112번째 식당
3113번째 식당
3114번째 식당
3115번째 식당
3116번째 식당
3117번째 식당
3118번째 식당
3119번째 식당
3120번째 식당
3121번째 식당
3122번째 식당
3123번째 식당
3124번째 식당
3125번째 식당
3126번째 식당
3127번째 식당
3128번째 식당
3129번째 식당




In [3]:
# dataframe 저장 및 csv 저장
review_df = pd.DataFrame(data_list) 
review_df.columns = ['UserID', 'Restaurant', 'Category', 'roadAddress', 'tel', 'rating', 'Timestamp'] 
review_df.to_csv('review.csv', encoding='utf-8-sig') 

# json 저장
import json 

file_path = "./user_review_id.json"
with open(file_path, 'w', encoding='utf-8') as outfile:
    json.dump(user_review_id, outfile, ensure_ascii=False)

file_path = "./review.json"
with open(file_path, 'w', encoding='utf-8') as outfile:
    json.dump(review_json, outfile, ensure_ascii=False)

file_path = "./image.json"
with open(file_path, 'w', encoding='utf-8') as outfile:
    json.dump(image_json, outfile, ensure_ascii=False)

file_path = "./menu.json"
with open(file_path, 'w', encoding='utf-8') as outfile:
    json.dump(image_json, outfile, ensure_ascii=False)